# Forecasting Consensus Expectations: Nonfarm Payrolls (NFP)

### Nicholas Wong

**Imports**

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

from tqdm.auto import tqdm
from scipy import stats, special
from scipy.optimize import brentq
from collections import defaultdict
from itertools import product
from scipy.stats import t as student_t, norm, binomtest, jarque_bera

In [ ]:
# read parquets
OUT_DIR = "../out"        
DF_FILE       = "df.parquet"
DF_FULL_FILE  = "df_full.parquet"

df       = pd.read_parquet(os.path.join(OUT_DIR, DF_FILE),      engine="pyarrow")
df_full  = pd.read_parquet(os.path.join(OUT_DIR, DF_FULL_FILE), engine="pyarrow")

print("df shape     :", df.shape)
print("df_full shape:", df_full.shape)

## Dynamic BMA

In [46]:
import numpy as np
import pandas as pd
from scipy.stats import t as student_t, norm
from itertools import product
from tqdm.auto import tqdm
from scipy.stats import binomtest

# ── PREP ─────────────────────────────────────────────────────────
# assume `df` is your COVID-filtered DataFrame with columns:
# ["release_date","economist","forecast","actual","error"]
df["release_date"] = pd.to_datetime(df["release_date"])
dates = np.sort(df["release_date"].unique())

# ── GRID SETTINGS ────────────────────────────────────────────────
windows = [6, 12, 24]            # contiguity windows (months)
lambdas = [0.80, 0.85, 0.90, 0.95]# forgetting factors
nu_grid = [3, 5, 10, 20, 50]     # Student-t degrees of freedom
ridge   = 1e-6                   # regularization to avoid div/0

results = []

# ── DYNAMIC BMA GRID SEARCH ─────────────────────────────────────
for window, lam, nu in tqdm(product(windows, lambdas, nu_grid),
                             desc="grid search",
                             total=len(windows)*len(lambdas)*len(nu_grid)):
    # initialize equal prior probs for all economists
    prev_post = None

    records = []
    for idx in range(window, len(dates)):
        t_date      = dates[idx]
        lookback    = dates[idx-window:idx]
        hist        = df[df["release_date"].isin(lookback)]
        # require full coverage over the window
        valid = hist.groupby("economist")["forecast"]\
                    .apply(lambda s: s.notna().all())
        econs = valid[valid].index
        if len(econs)==0:
            continue

        # extract prior
        if prev_post is None:
            prior = pd.Series(1.0, index=econs)
        else:
            # apply forgetting factor to last posterior, restrict to current econs
            prior = prev_post.reindex(econs, fill_value=0.0) ** lam
        prior = prior / prior.sum()

        # compute log-likelihood this period under Student-t
        loglik = {}
        for econ in econs:
            errs = hist.loc[hist["economist"]==econ, "error"].dropna().values
            if len(errs)==0:
                continue
            sigma = np.std(errs, ddof=1)
            loglik[econ] = student_t.logpdf(errs, df=nu, loc=0, scale=sigma).sum()

        if not loglik:
            continue

        # posterior ∝ prior * likelihood  (in log space for stability)
        e_list = list(loglik.keys())
        logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])
        w_unn  = np.exp(logw - logw.max())
        post   = pd.Series(w_unn, index=e_list)
        post  /= post.sum()

        # forecast at t_date
        cur = df[(df["release_date"]==t_date)&(df["economist"].isin(post.index))]
        f_t = cur.set_index("economist")["forecast"].dropna()
        w   = post.reindex(f_t.index).fillna(0.0)
        w  /= w.sum()
        if w.sum()==0:
            continue

        yhat_smart  = (w * f_t).sum()
        yhat_median = df.loc[df["release_date"]==t_date, "forecast"]\
                         .dropna().median()
        y_actual    = df.loc[df["release_date"]==t_date, "actual"].iloc[0]

        records.append((t_date, yhat_smart, yhat_median, y_actual))
        prev_post = post  # carry forward for next period

    # assemble OOS table & metrics
    if not records:
        continue
    oos = pd.DataFrame(records, columns=["date","smart","median","actual"])
    oos["smart_err"]  = oos["smart"]  - oos["actual"]
    oos["median_err"] = oos["median"] - oos["actual"]
    # directional
    oos["actual_dir"] = (oos["actual"]>oos["median"]).astype(int)
    oos["pred_dir"]   = (oos["smart"]>oos["median"]).astype(int)

    obs         = len(oos)
    rmse_smart  = np.sqrt((oos["smart_err"]**2).mean())
    rmse_median = np.sqrt((oos["median_err"]**2).mean())
    # Diebold-Mariano (MSE)
    d        = oos["smart_err"]**2 - oos["median_err"]**2
    dm_stat  = d.mean()/d.std(ddof=1)*np.sqrt(obs)
    dm_p     = 2*(1 - norm.cdf(abs(dm_stat)))
    # hit‐rate tests
    hits     = (oos["actual_dir"]==oos["pred_dir"]).astype(int)
    hit_rt   = hits.mean()
    binom_p  = binomtest(hits.sum(), obs, 0.5).pvalue
    p1, p2   = oos["pred_dir"].mean(), oos["actual_dir"].mean()
    c_joint  = (oos["pred_dir"] & oos["actual_dir"]).mean()
    pt_stat  = (c_joint - p1*p2)/np.sqrt(p1*p2*(1-p1)*(1-p2)/obs)
    pt_p     = 2*(1 - norm.cdf(abs(pt_stat)))

    results.append({
        "window":      window,
        "lambda":      lam,
        "nu":          nu,
        "obs":         obs,
        "RMSE_smart":  rmse_smart,
        "RMSE_median": rmse_median,
        "HitRate":     hit_rt,
        "Binom_p":     binom_p,
        "PT_stat":     pt_stat,
        "PT_p":        pt_p,
        "DM_stat":     dm_stat,
        "DM_p":        dm_p
    })

# final DataFrame
res_df = pd.DataFrame(results).sort_values("RMSE_smart")
print(res_df.to_string(index=False, float_format="{:.3f}".format))

C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["release_date"] = pd.to_datetime(df["release_date"])


grid search:   0%|          | 0/60 [00:00<?, ?it/s]

C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning: divide by zero encountered in log
  logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])
C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning: divide by zero encountered in log
  logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])
C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning: divide by zero encountered in log
  logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])
C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning: divide by zero encountered in log
  logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])
C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning: divide by zero encountered in log
  logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])
C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning

 window  lambda  nu  obs  RMSE_smart  RMSE_median  HitRate  Binom_p  PT_stat  PT_p  DM_stat  DM_p
     24   0.800  10  245      82.039       74.767    0.482    0.609   -0.651 0.515    3.941 0.000
     24   0.800  20  245      82.040       74.767    0.482    0.609   -0.638 0.523    3.944 0.000
     24   0.900  50  245      82.052       74.767    0.465    0.307   -1.139 0.255    3.946 0.000
     24   0.850  50  245      82.053       74.767    0.478    0.523   -0.760 0.447    3.941 0.000
     24   0.800  50  245      82.057       74.767    0.478    0.523   -0.760 0.447    3.954 0.000
     24   0.850  20  245      82.062       74.767    0.478    0.523   -0.760 0.447    3.943 0.000
     24   0.800   5  245      82.071       74.767    0.482    0.609   -0.651 0.515    3.956 0.000
     24   0.900  20  245      82.079       74.767    0.465    0.307   -1.139 0.255    3.956 0.000
     24   0.850  10  245      82.080       74.767    0.469    0.371   -1.017 0.309    3.952 0.000
     24   0.900  10 

C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning: divide by zero encountered in log
  logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])
C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning: divide by zero encountered in log
  logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])
C:\Users\nwong\AppData\Local\Temp\ipykernel_252\2933347072.py:63: RuntimeWarning: divide by zero encountered in log
  logw   = np.array([np.log(prior[e]) + loglik[e] for e in e_list])



## Distribution Forecasting via Gaussian Mixture Model

Build Gaussian-mixture predictive distribution for each release.

**Motivation**<br>
Instead of point forecasting, we can use a gaussian mixture model to forecast a probability distribution 

**Background**<br>
Briefly: a Gaussian mixture model treats each forecaster as a mini-model whose uncertainty is normal. 
- Each economist represents a tiny probabilistic model. Point forecast tells us where economist i thinks payrolls will land. Recent RMSE tells us how wide a credible band we should give them. 
- Decide how much weight to give each bell via inverse MSE weighting 
- Blend the bells (Add each normal)
- When we have this mapped density, we can extract: CDF (integration), get quantiles, get tail risks, draw random samples

Here, we build the Gaussian-mixture model with the same 6 month contiguity and inverse-MSE weighting scheme.

In [ ]:
# df_filtered

,release_date,period,actual,economist,firm,forecast,asof,error,median_forecast_x,median_forecast_y,median_forecast
0,2000-02-04,2000-01-31,387.0,Adam Chester,Lloyds Bank PLC,NaN,NaT,NaN,267.5,267.5,267.5
1,2000-02-04,2000-01-31,387.0,Alessandro Truppia,Aletti Gestielle Sgr Spa,NaN,NaT,NaN,267.5,267.5,267.5
2,2000-02-04,2000-01-31,387.0,Alexandre De Azara,Banco UBS SA,NaN,NaT,NaN,267.5,267.5,267.5
3,2000-02-04,2000-01-31,387.0,Alison Lynn Reaser,Point Loma Nazarene University,NaN,NaT,NaN,267.5,267.5,267.5
4,2000-02-04,2000-01-31,387.0,Allan Von Mehren,Danske Bank AS,NaN,NaT,NaN,267.5,267.5,267.5
...,...,...,...,...,...,...,...,...,...,...,...
96746,2025-06-06,2025-05-01,139.0,Uwe Duerkop,Berliner Sparkasse,75.0,2025-06-02,-64.0,126.0,126.0,126.0
96747,2025-06-06,2025-05-01,139.0,William Adams,Comerica Bank,125.0,2025-06-02,-14.0,126.0,126.0,126.0
96748,2025-06-06,2025-05-01,139.0,Yongxin Chen,Japan Post Insurance Co Ltd,150.0,2025-05-23,11.0,126.0,126.0,126.0
96749,2025-06-06,2025-05-01,139.0,Yonie Fanning,Mizrahi Tefahot Bank Limited,98.0,2025-06-05,-41.0,126.0,126.0,126.0


In [ ]:
# df = df_filtered

In [ ]:
# dates = np.sort(df["release_date"].unique())

# mixtures = {}

# for idx in range(6, len(dates)):
#     t = dates[idx]
#     window = dates[idx-6:idx]
    
#     # slice six month history
#     hist = df[df["release_date"].isin(window)]
    
#     # contiguity filter 
#     valid = hist.groupby("economist")["forecast"].apply(lambda s: s.notna().all())
#     valid_econ = valid[valid].index
#     if valid_econ.empty:
#         continue
    
#     # compute MSE over the window 
#     mse = (hist[hist["economist"].isin(valid_econ)]
#            .groupby("economist")["error"]
#            .apply(lambda s: np.nanmean((s.dropna())**2))
#            .dropna())
    
#     if mse.empty:
#         continue
    
#     # inverse mse weighting
#     w = 1.0/(mse + 1e-6)
#     w = w/w.sum()
    
#     # get current forecasts for t, align + renormalize
#     cur = df[(df["release_date"] == t) & 
#              (df["economist"].isin(valid_econ))]
#     f_t = cur.set_index("economist")["forecast"].dropna()
#     w = w.loc[w.index.isin(f_t.index)]
    
#     if w.empty:
#         continue
    
#     w = w/w.sum()
    
#     # mixture params
#     mixtures[t] = {
#         "weights": w.values,
#         "means": f_t.loc[w.index].values,
#         "vars": mse.loc[w.index].values
#     }

In [ ]:
# #--------------------
# # Helper function: get CDF at x 
# #--------------------

# def mixture_cdf(x, weights, means, variances):
#     """
#     Compute CDF of a Gaussian mixture at value x.
#     """
#     sigmas = np.sqrt(variances)
#     return np.sum(weights * norm.cdf(x, loc=means, scale = sigmas))

In [ ]:
# #--------------------
# # Quantile function for a given date
# #--------------------

# def get_quantiles(mixtures, date, probs):
#     """
#     Return quantiles for 'date' at probabilities in 'probs'
#     @param mixtures: dict as built by GMM code, mixtures[date] = {"weights, "means", "vars"}
#     @param date: Release date of target
#     @param prob: sequence of float representing quantile levels (e.g. [0.25, 0.5, 0.75])
#     """
#     params = mixtures[date]
#     w, m, v = params["weights"], params["means"], params["vars"]
    
#     # bounds for root finding
#     delta = 5 * np.sqrt(v.max())
#     lo = m.min() - delta
#     hi = m.max() + delta
    
#     def find_q(p):
#         return brentq(lambda x: mixture_cdf(x, w, m, v) - p, lo, hi)
    
#     q_vals = [find_q(p) for p in probs]
    
#     return pd.Series(q_vals, index=probs)
    

In [ ]:
# #--------------------
# # Tail probabilities 
# #--------------------

# def get_tail_probs(mixtures, date, thresholds):
#     """
#     Return P(Y <= threshold) and P(Y >= threshold) for each threshold. 
#     @param mixtures: dictionary built by GMM, as above
#     @param date: release date
#     @thresholds: sequence of float representing jobs (k) levels to compute tail probabilities 
#     """
#     params = mixtures[date]
#     w, m, v = params["weights"], params["means"], params["vars"]
    
#     p_le = [mixture_cdf(t, w, m, v) for t in thresholds]
#     return pd.DataFrame({
#         "threshold": thresholds,
#         "P_le": p_le,
#         "P_ge": [1-p for p in p_le]
#     })

In [ ]:
# plot_date = dates[-1]           # let's try distribution forecasting for the most recent relase (June 6)


# # Quartiles 
# quartiles = get_quantiles(mixtures, plot_date, probs=[0.25, 0.5, 0.75])

# print("Quartiles:\n", quartiles)

Quartiles:
 0.25     92.095180
0.50    135.986634
0.75    178.316179
dtype: float64


In [ ]:
# # Deciles 

# deciles = get_quantiles(mixtures, plot_date, probs=np.arange(0.1, 1.0, 0.1))
# print("Deciles:\n", deciles)

Deciles:
 0.1     51.168651
0.2     80.973923
0.3    101.998029
0.4    119.685524
0.5    135.986634
0.6    152.060576
0.7    169.017917
0.8    188.613970
0.9    215.651560
dtype: float64


In [ ]:
# # Tail probabilities: custom thresholds 

# tails = get_tail_probs(mixtures, plot_date, thresholds=[0, 100, 200, 300])
# print("Tail probabilities:\n", tails)

Tail probabilities:
    threshold      P_le      P_ge
0          0  0.022624  0.977376
1        100  0.289523  0.710477
2        200  0.847797  0.152203
3        300  0.994251  0.005749


**DM Test for Mixture Model**

Run a Diebold-Mariano test on log-score differentials

In [ ]:
# # Benchmark sigma 
# sigma_med = np.sqrt(np.mean(oos["median_err"]**2))

# # Collect log-score for smart (GMM) vs. benchmark 
# ls_smart, ls_bench, ds = [], [], []

# for t, params in mixtures.items():
#     # actual print 
#     y = df.loc[df["release_date"] == t, "actual"].iloc[0]
    
#     # smart density at y 
#     w = params["weights"]
#     m = params["means"]
#     v = params["vars"]
    
#     dens_smart = np.sum(w * norm.pdf(y, loc=m, scale=np.sqrt(v)))
#     ls_smart.append(np.log(dens_smart))
    
#     # benchmark density at y (Normal with median forecast & sigma_med)
#     mu_b = df.loc[df["release_date"] == t, "median_forecast"].iloc[0]
#     dens_bench = norm.pdf(y, loc=mu_b, scale = sigma_med)
#     ls_bench.append(np.log(dens_bench))
    
#     # record date
#     ds.append(t)

# scores = pd.DataFrame({
#     "date":     ds, 
#     "ls_smart":       ls_smart,
#     "ls_bench":         ls_bench
# })
# scores["d"] = scores["ls_smart"] - scores["ls_bench"]   # score differential 

# # DM test on log-score logg 
# d = scores["d"]
# dm_stat = d.mean() / d.std(ddof=1) * np.sqrt(len(d))
# p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))

# print("DM on log-predictive scores")
# print(f"DM statistic: {dm_stat:.3f}")
# print(f"p-value : {p_value: .4f}")

DM on log-predictive scores
DM statistic: -0.692
p-value :  0.4891
